<a href="https://colab.research.google.com/github/bffarinha/Melanoma_PFG/blob/main/PFG_melanoma_version_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Melanoma detection in dermatoscopic images using contextual information and Convolutional Neural Networks.
By Brenda Farinha Fernandes

November 2022



*   Trained using TensorFlow Keras 2.11 on Google Colaboratory TPUs
*   All images are contained within a TFRecord
*   All TFRecords contain similar proportions of benign/malign cases
*   Images in TFRecord format have been resized to a uniform 1024x1024.



## Setup

### Required libraries

In [1]:
import zipfile
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from functools import partial

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 

import tensorflow as tf
print('Tensorflow version ', tf.__version__)

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Tensorflow version  2.11.0
Mounted at /content/drive


### TPU or GPU Distribution



In [2]:
# Detect hardware 
try: 
  # TPU detection
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU', tpu.master())
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
  print('Not connect to TPU')

# Select appropriate distribution strategy 
if tpu: 
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')

AUTOTUNE = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print('REPLICAS:', REPLICAS) 

Running on TPU grpc://10.75.131.50:8470
Running on TPU  ['10.75.131.50:8470']
REPLICAS: 8


### Global parameters

In [3]:
BATCH_SIZE = 16 * REPLICAS
BUFFER_SIZE = 2048
EPOCHS = 100
IMAGE_SIZE = [256, 256]

print("BATCH SIZE:", BATCH_SIZE)
print("BUFFER SIZE:", BUFFER_SIZE)
print("EPOCHS", EPOCHS)
print("IMAGE SIZE:", IMAGE_SIZE[0],"x",IMAGE_SIZE[1])

BATCH SIZE: 128
EPOCHS 100
IMAGE SIZE: 256 x 256


Paths of dataset

In [4]:
PATH_ds_img_jpeg_zip = '/content/drive/MyDrive/PFG_MELANOMA/ISIC_2020_Training_JPEG.zip'
PATH_ds_img_tfrecords_zip = '/content/drive/MyDrive/PFG_MELANOMA/tfrecords.zip'

PATH_ds_tfrecords = '/content/drive/MyDrive/PFG_MELANOMA/tfrecords'
PATH_ds_jpeg = '/content/drive/MyDrive/PFG_MELANOMA/images'

PATH_ds_train = '/content/drive/MyDrive/PFG_MELANOMA/ISIC_2020_Training_GroundTruth.csv'

GCS_PATH_TRAIN = 'gs://ds-tfrecord/train-tfrecord/train*.tfrec'

## Load dataset

Read [SIIM-ISIC 2020 Melanoma Classification Challenge Dataset](https://doi.org/10.34970/2020-ds01) 

Images are also provided in JPEG and TFRecord format (in the jpeg and tfrecords directories, respectively). Images in TFRecord format have been resized to a uniform 1024x1024.

Metadata is also provided outside of the DICOM format, in CSV files with this description:

* `image_name` - unique identifier, points to filename of related DICOM image
* `patient_id` - unique patient identifier
* `sex` - the sex of the patient (when unknown, will be blank)
* `age_approx` - approximate patient age at time of imaging
* `anatom_site_general_challenge` - location of imaged site
* `diagnosis` - detailed diagnosis information
* `benign_malignant` - indicator of malignancy of imaged lesion
* `target` - binarized version of the target variable



In [5]:
# Unzip dataset from Google Drive
# For TPU version not use 
dataset_unzip = False
if dataset_unzip: 
  zipfile.ZipFile(PATH_ds_img_jpeg_zip).extractall(PATH_ds_jpeg)
  zipfile.ZipFile(PATH_ds_img_tfrecords_zip).extractall(PATH_ds_tfrecords)

In [6]:
# Read dataset CSV with metadata information
train = pd.read_csv(f'{PATH_ds_train}')
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [8]:
# Count images
def count_data_images(path):
  # The number of data images is written in the name of the .tfrec files. Example: train10-2071.tfrec = 2071 data items
  n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in path]
  return np.sum(n)

In [7]:
# Split dataset for training and validation 
train_files, val_files = train_test_split(tf.io.gfile.glob(GCS_PATH_TRAIN), test_size=0.1, random_state=42, shuffle=True)

print("Number of training files = ", len(train_files))
print("Number of validation files = ", len(val_files))

Number of training files =  14
Number of validation files =  2


In [16]:
num_training_images = count_data_images(train_files)
num_validation_images = count_data_images(val_files)

STEPS_PER_EPOCH_TRAIN = num_training_images // BATCH_SIZE
STEPS_PER_EPOCH_VAL = num_validation_images // BATCH_SIZE

print("Number of training Images =", num_training_images)
print("Numer of steps per epoch in Train =", STEPS_PER_EPOCH_TRAIN)
print("\nNumber of validation Images =", num_validation_images)
print("Numer of steps per epoch in Train =", STEPS_PER_EPOCH_VAL)

Number of training Images = 28984
Numer of steps per epoch in Train = 226

Number of validation Images = 4142
Numer of steps per epoch in Train = 32


## Data preproccesing

In [9]:
def decode_image(image):

  image = tf.image.decode_jpeg(image, channels = 3)
  image = tf.cast(image, tf.float32) / 255.0
  image = tf.image.resize(image, IMAGE_SIZE)

  return image

In [10]:
def read_labeled_tfrecord(example, labeled = False):

  # Create a description of the features.
  if labeled: 
    TFREC_FORMAT = {
        "image"       : tf.io.FixedLenFeature([], tf.string),
        "target"      : tf.io.FixedLenFeature([], tf.int64)     
    }
  else: 
    TFREC_FORMAT = {
        "image"       : tf.io.FixedLenFeature([], tf.string),
        "image_name"  : tf.io.FixedLenFeature([], tf.string)
    }

  example = tf.io.parse_single_example(example, TFREC_FORMAT)
  
  image = decode_image(example["image"])
  
  if labeled: 
    label = tf.cast(example["target"], tf.int32)
    return image, label
  else:
    image_name = example["image_name"]
    return image, image_name   

In [11]:
def image_augmentation(image, label):

  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_hue(image, 0.025)
  image = tf.image.random_saturation(image, 0.6, 1.4)
  image = tf.image.random_contrast(image, 0.7, 1.4)
  image = tf.image.random_brightness(image, 0.1)

  return image, label

[tf.data.TFRecordDataset Methods](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset#methods_2)

* `tf.data.Options():` An Options object can be, for instance, used to control which graph optimizations to apply.

* `.experiemental_deterministic:` experimental_deterministic refers to whether the outputs need to be produced in deterministic order. If None, defaults to True. Here, the data is unordered, hence we don't need to process it in an order which may slow down our speed.

* `cache():` Caches the elements in this dataset. The first time the dataset is iterated over, its elements will be cached either in the specified file or in memory. Subsequent iterations will use the cached data.

* `repeat():` Repeats this dataset so each original value is seen count times.

* `shuffle():` Randomly shuffles the elements of this dataset. This dataset fills a buffer with `buffer_size` elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.

* `map():` Maps map_func across the elements of this dataset. This transformation applies map_func to each element of this dataset, and returns a new dataset containing the transformed elements, in the same order as they appeared in the input. 

In [12]:
def load_dataset(files, ordered = False, labeled = False, repeat = False, cache = False, augment = False): 

  ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTOTUNE)

  if not ordered: 
    ds = ds.shuffle(1024)
    options = tf.data.Options()
    options.experimental_deterministic = False
    ds = ds.with_options(options)
  
  ds = ds.map(partial(read_labeled_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
  #ds = ds.map(lambda example: read_labeled_tfrecord(example, labeled), num_parallel_calls=AUTOTUNE)

  if augment: 
    ds = ds.map(image_augmentation, num_parallel_calls=AUTOTUNE)

  if repeat: 
    ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  if cache: 
    ds = ds.cache()

  ds = ds.prefetch(AUTOTUNE)

  return ds  

In [13]:
train_ds = load_dataset(train_files, ordered = False, labeled = True, repeat = True, cache = False, augment = True)
val_ds = load_dataset(val_files, ordered = False, labeled = True, repeat = False, cache = True, augment = False)

In [14]:
#ds = tf.data.TFRecordDataset(train_files, num_parallel_reads=AUTOTUNE)

In [15]:
#raw_example = next(iter(ds))
#parsed = tf.train.Example.FromString(raw_example.numpy())

#parsed.features

## Model fit

### [Calculate class weights ](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights)

revisar pq estos no son los pesos correctos. dividi el train en val y train. sacar este dato de los tfrecord


In [17]:
malignant = len(train[train["target"] == 1])
benign = len(train[train["target"] == 0 ])
total = len(train) 

print("Malignant Cases in Train Data = ", malignant)
print("Benign Cases In Train Dataset = ",benign)
print("Total Cases In Train Dataset = ",total)
print("Ratio of Malignant to Benign = ",malignant/benign)

Malignant Cases in Train Data =  583
Benign Cases In Train Dataset =  32542
Total Cases In Train Dataset =  33125
Ratio of Malignant to Benign =  0.017915309446254073


In [18]:
weight_malignant = (total/malignant)/2.0
weight_benign = (total/benign)/2.0

class_weight = {0 : weight_benign , 1 : weight_malignant}

print("Weight for benign cases = {:.2f} " .format(class_weight[0]))
print("Weight for malignant cases = {:.2f} " .format(class_weight[1]))

Weight for benign cases = 0.51 
Weight for malignant cases = 28.41 


Defining Callbacks

*   `EarlyStopping`: Stop training when a monitored metric has stopped improving.
*   `ModelCheckpoint`: Callback to save the Keras model or model weights at some frequency.



In [19]:
def get_lr_callback():

  callback_early_stopping = tf.keras.callbacks.EarlyStopping(
      patience = 15, 
      verbose = 0, 
      restore_best_weights = True)

  callbacks_lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(
      monitor = "val_auc", 
      factor = 0.1, 
      patience = 10,
      verbose = 0, 
      min_lr = 1e-6)

  callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
      filepath="melanoma_detection_weights.h5",
      save_weights_only=True,
      monitor='val_auc',
      mode='max',
      save_best_only=True,
      verbose=1)
  
  return [callback_early_stopping, callbacks_lr_reduce, callback_checkpoint]

In [20]:
def make_model():
  with strategy.scope():

    bias = np.log(malignant/benign)
    bias = tf.keras.initializers.Constant(bias)
  
    input = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

    cnn_model = tf.keras.applications.EfficientNetB6(
        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )(input)

    x = tf.keras.layers.GlobalAveragePooling2D()(cnn_model) 

    output = tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=bias)(x)

    model = tf.keras.Model(inputs=input, outputs=output, name='aNetwork')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.05),
        metrics = [tf.keras.metrics.AUC(name='auc')]
    )

    model.summary()
  
  return model

In [21]:
initial_model = make_model()

initial_model.fit(
    train_ds, 
    epochs=EPOCHS,
    verbose=True, 
    steps_per_epoch=STEPS_PER_EPOCH_TRAIN, 
    validation_data=val_ds, 
    validation_steps=STEPS_PER_EPOCH_VAL,
    callbacks=get_lr_callback(),
    class_weight = class_weight
)

Model: "aNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetb6 (Functional)  (None, 8, 8, 2304)       40960143  
                                                                 
 global_average_pooling2d (G  (None, 2304)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2305      
                                                                 
Total params: 40,962,448
Trainable params: 40,738,009
Non-trainable params: 224,439
_________________________________________________________________
Epoch 1/100
226/226 [==============================] - ETA: 0s - loss: 0.7160 - auc: 0.6937
Epoch 1: val_a

In [22]:
initial_weights = "melanoma_detection_weights.h5"
initial_model.save_weights(initial_weights)

In [ ]:
#weighted_model = make_model()
#weighted_model.load_weights(initial_weights)

### Evaluate metrics